# TODO
- interpolate missing values

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.metrics import mean_squared_error
import xgboost as xgb
import sys
sys.path.append('../')
import lib.tools as tools

pd.options.display.max_columns = 50
raw_path = '../data/raw/'

IndentationError: unindent does not match any outer indentation level (tools.py, line 48)

In [14]:
model = xgb.XGBRegressor()


In [5]:
# Batch
X = np.arange(17)
y = np.zeros(17)
from sklearn.model_selection import TimeSeriesSplit
tssplit = TimeSeriesSplit(n_splits=4)
for train_index, test_index in tssplit.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [0 1 2 3 4 5 6 7] TEST: [ 8  9 10]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10] TEST: [11 12 13]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] TEST: [14 15 16]
